In [1]:
import torch
import numpy as np
from models.training import easyTrainer, weights_to_dataset
from models.nODE import nODE, make_nODE_from_parameters
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchdiffeq import odeint
from data_creation import create_dataset, torch_parameter_structure

In [2]:
x_train, x_noise, y_train, param = create_dataset(2,1,100)

In [6]:
class param_classifier(torch.nn.Module):
    def __init__(self, input_size, ode_dim, layers_size=[10, 10], device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')):
        super().__init__()

        self.layers_size = layers_size
        self.input_size = input_size
        self.output_size = 2 * ode_dim ** 2 + 3 * ode_dim
        # print(self.output_size)
        self.ode_dim = ode_dim

        self.num_layers = len(layers_size) + 1
        self.device = device

        # initialise linear layers for classification block
        self.linears = torch.nn.ModuleList([torch.nn.Linear(input_size, layers_size[0])])
        self.linears.extend(
            [torch.nn.Linear(layers_size[i - 1], layers_size[i]) for i in range(1, self.num_layers - 1)])
        self.linears.append(torch.nn.Linear(layers_size[-1], self.output_size))
        return
    

    def forward_integration(self, x, parameter, integration_time=None):
        if integration_time is None:
            time_intervals = torch.tensor([0., 1.])
        else:
            time_intervals = torch.tensor(integration_time)
        integration_interval = torch.tensor([0,1.]).float().type_as(x)

        par_struct = torch_parameter_structure(self.ode_dim)
        par_struct.set_vec_par(parameter)
        gamma, Win, bin, Wout, bout = par_struct.get_parameters()
        
        dt = 0.01
        out = odeint(lambda t, x : self.right_hand_side(t, x, Win, Wout, bin, bout, gamma), x, integration_interval, method='euler', options={'step_size': dt})
        return out[-1, :]
               
    def right_hand_side(self, t, x, Win,Wout,bin,bout,gamma):
        out = x.matmul(Win.t()) + bin.t()
        out = out.matmul(Wout.t()) + bout.t()
        out = x.matmul(gamma) + out
        return out

    def get_adjacency(self,parameter):
        Win = torch.zeros(self.ode_dim,self.ode_dim).to(self.device)
        Wout = torch.zeros(self.ode_dim,self.ode_dim).to(self.device)

        k=0

        for i in range(0,self.ode_dim):
            for j in range(0,self.ode_dim):
                Win[i][j] = parameter[k]
                k += 1

        for i in range(0,self.ode_dim):
            for j in range(0,self.ode_dim):
                Wout[i][j] = parameter[k]
                k += 1
        
        A = Wout.matmul(Win).flatten()
        return A

    # forward pass of NN (both classifier and neural ODE)
    def forward(self, data):
        x = data
        for i in range(0, self.num_layers):
            x = self.linears[i](x)
            if i < self.num_layers-1:
                x = F.relu(x)

        # here x denote the estimated parameters for the ODE
        # x = self.linears[len(self.layers_size) - 1](x)

        return x

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

classifier = param_classifier(8, 2).to(device)

x = x_train[0][0].to(device)

out = classifier(x)

u0 = x[:2]

print(out)

ut_hat = classifier.forward_integration(u0, out)

print(ut_hat)



tensor([ 0.1677, -0.2346,  0.0854, -0.0723, -0.3763,  0.2523, -0.2670,  0.1658,
        -0.0280, -0.2279, -0.1442,  0.0450,  0.3542, -0.3352], device='cuda:0',
       grad_fn=<AddBackward0>)
tensor([-0.0660,  0.6840], device='cuda:0', grad_fn=<SliceBackward0>)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

classifier = param_classifier(8, 2).to(device)
ode_dim = 2
integration_time = 1
# node = nODE(ode_dim, architecture='both', time_interval=[0, integration_time]).to(device)
# # replaced by classifier.neuralODE
loss_fn = torch.nn.MSELoss()

optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-2)
"""
for name, param in classifier.named_parameters():
    print(name, param.requires_grad)

for name, param in node.named_parameters():
    print(name, param.requires_grad)
"""
for epoch in range(100):
    optimizer.zero_grad()
    loss = 0

    for i in range(0, len(x_train)):
        x = x_train[i][0].to(device)
        y = y_train[i][0].to(device)

        # get necessary inputs from data
        classifier_inp = x

        # get u_0, u_T from data
        u0 = classifier_inp[:2]
        ut = y[:2]

        # get true adjacency from data
        A = y[2:]

        # estimate parameters using classifier network
        p = classifier(classifier_inp)

        # integrate ODE
        ut_hat = classifier.forward_integration(u0, p)

        # get adjacency matrix
        A_hat = classifier.get_adjacency(p)

        # add to loss: error between found solution at time t and true solution
        loss = loss + loss_fn(ut_hat.float(), ut.float())

        # add to loss: difference between estimated network and true network
        loss = loss + 0.01*loss_fn(A_hat.float(),A.float())

        loss += loss_fn(p, 0*p)
    print('Epoch ' + str(epoch))
    print(loss)

    # backward propagation
    loss.backward(retain_graph=True)
    for name, param in classifier.named_parameters():
        if param.grad is None:
            print(f"Gradient for {name} is None")
        #else:
        #    # print(f"Gradient for {name}: {param.grad}")
    optimizer.step()
    # print(list(classifier.parameters())[0].grad)


Epoch 0
tensor(126.9780, device='cuda:0', grad_fn=<AddBackward0>)
Epoch 1
tensor(120.8330, device='cuda:0', grad_fn=<AddBackward0>)
Epoch 2
tensor(115.9976, device='cuda:0', grad_fn=<AddBackward0>)
Epoch 3
tensor(111.9706, device='cuda:0', grad_fn=<AddBackward0>)
Epoch 4
tensor(108.4441, device='cuda:0', grad_fn=<AddBackward0>)
Epoch 5
tensor(105.1626, device='cuda:0', grad_fn=<AddBackward0>)
Epoch 6
tensor(102.0072, device='cuda:0', grad_fn=<AddBackward0>)
Epoch 7
tensor(98.9031, device='cuda:0', grad_fn=<AddBackward0>)
Epoch 8
tensor(95.8467, device='cuda:0', grad_fn=<AddBackward0>)
Epoch 9
tensor(92.8192, device='cuda:0', grad_fn=<AddBackward0>)
Epoch 10
tensor(89.8035, device='cuda:0', grad_fn=<AddBackward0>)
Epoch 11
tensor(86.8106, device='cuda:0', grad_fn=<AddBackward0>)
Epoch 12
tensor(83.7060, device='cuda:0', grad_fn=<AddBackward0>)
Epoch 13
tensor(80.5245, device='cuda:0', grad_fn=<AddBackward0>)
Epoch 14
tensor(77.2901, device='cuda:0', grad_fn=<AddBackward0>)
Epoch 15
tens

In [14]:
correct = 0
total = 0
for data_point in range(0, len(x_train)):
    x = x_train[i][0].to(device)
    y = y_train[i][0].to(device)
    out = classifier(x)

    print(classifier.get_adjacency(out))

    print(out)

    print(y[2:])

    # guess = out.cpu() > 0.7
    # print(guess)
    # print(data_point[1])
    # if sum(abs(guess.float()-data_point[1])) < 0.001:
    #     correct += 1
    # total += 1

#print(correct/total)

tensor([ 0.3125,  0.2708, -0.4880, -0.1950], device='cuda:0',
       grad_fn=<ReshapeAliasBackward0>)
tensor([ 0.2149,  0.5206, -0.7435, -0.2396,  0.3768, -0.3114, -0.0837,  0.6321,
         0.0469, -0.1967,  1.0317, -0.8236,  0.1713,  0.0347], device='cuda:0',
       grad_fn=<AddBackward0>)
tensor([ 1.,  0., -1.,  0.], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([ 0.3125,  0.2708, -0.4880, -0.1950], device='cuda:0',
       grad_fn=<ReshapeAliasBackward0>)
tensor([ 0.2149,  0.5206, -0.7435, -0.2396,  0.3768, -0.3114, -0.0837,  0.6321,
         0.0469, -0.1967,  1.0317, -0.8236,  0.1713,  0.0347], device='cuda:0',
       grad_fn=<AddBackward0>)
tensor([ 1.,  0., -1.,  0.], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([ 0.3125,  0.2708, -0.4880, -0.1950], device='cuda:0',
       grad_fn=<ReshapeAliasBackward0>)
tensor([ 0.2149,  0.5206, -0.7435, -0.2396,  0.3768, -0.3114, -0.0837,  0.6321,
         0.0469, -0.1967,  1.0317, -0.8236,  0.1713,  0.0347], device='cuda:0',
       g